### What is Non-Maximum Suppression?

Non-Maximum Suppression is a technique used to select the most appropriate bounding boxes and remove overlapping duplicates in object detection tasks. It's particularly important when a model detects multiple bounding boxes for the same object.

### How NMS Works

1. **Input**:
   - A list of detected bounding boxes for the same box.
   - Corresponding confidence scores for each box.
   - An IoU (Intersection over Union) threshold

### Process

In [ ]:
import torch

def intersection_over_union(predictions,labels,format):
    '''
    Calculates intersection over union 

    Parameters:
    predictions: predictions of bounding boxes
    lables: correct labels of boxes
    format: midpoint/corners (x,y,w,h) or (x1,y1,x2,y2)
    '''

    if format == "midpoint":
        box1_x1 = predictions[..., 0:1] - predictions[..., 2:3] / 2
        box1_y1 = predictions[..., 1:2] - predictions[..., 3:4] / 2
        box1_x2 = predictions[..., 0:1] + predictions[..., 2:3] / 2
        box1_y2 = predictions[..., 1:2] + predictions[..., 3:4] / 2
        box2_x1 = labels[..., 0:1] - labels[..., 2:3] / 2
        box2_y1 = labels[..., 1:2] - labels[..., 3:4] / 2
        box2_x2 = labels[..., 0:1] + labels[..., 2:3] / 2
        box2_y2 = labels[..., 1:2] + labels[..., 3:4] / 2

    elif format == "corners":
        box1_x1 = predictions[..., 0:1]
        box1_y1 = predictions[..., 1:2]
        box1_x2 = predictions[..., 2:3]
        box1_y2 = predictions[..., 3:4]
        box2_x1 = labels[..., 0:1]
        box2_y1 = labels[..., 1:2]
        box2_x2 = labels[..., 2:3]
        box2_y2 = labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # clamp(0) for boxes with no intersection  
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

def nms(bboxes, iou_threshold, threshold, box_format="corners"):
    """
    Does Non Max Suppression given bboxes

    Parameters:
        bboxes (list): list of lists containing all bboxes with each bboxes
        specified as [class_pred, prob_score, x1, y1, x2, y2]
        iou_threshold (float): threshold where predicted bboxes is correct
        threshold (float): threshold to remove predicted bboxes (independent of IoU) 
        box_format (str): "midpoint" or "corners" used to specify bboxes

    Returns:
        list: bboxes after performing NMS given a specific IoU threshold
    """

    assert type(bboxes) == list

    bboxes = [box for box in bboxes if box[1] > threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []

    while bboxes:
        chosen_box = bboxes.pop(0)

        bboxes = [
            box
            for box in bboxes
            if box[0] != chosen_box[0]
            or intersection_over_union(
                torch.tensor(chosen_box[2:]),
                torch.tensor(box[2:]),
                box_format=box_format,
            )
            < iou_threshold
        ]

        bboxes_after_nms.append(chosen_box)

    return bboxes_after_nms

3. **Steps**:
   - Sort all bounding boxes by their confidence scores (highest to lowest)
   - Select the box with the highest score
   - Compare this box with all remaining boxes using IoU
   - Remove boxes that overlap with the selected box above the IoU threshold
   - Repeat the process with the remaining boxes



This code shows how NMS principles are applied in practice, where multiple detections of the same license plate are handled to avoid redundant detections.

### Benefits of NMS

1. **Reduces Redundancy**: Eliminates multiple detections of the same object
2. **Improves Accuracy**: Keeps the most confident detection for each object
3. **Cleaner Output**: Produces cleaner, more interpretable results
4. **Computational Efficiency**: Reduces the number of boxes that need to be processed downstream